In [1]:
import sys
import logging
import argparse
import numpy as np

sys.path.insert(0, '../../../network')
from network import Population, RateNetwork
from transfer_functions import ErrorFunction
from connectivity import SparseConnectivity, LinearSynapse
from sequences import GaussianSequence

In [3]:
T = 0.25

phi = ErrorFunction(mu=0.22, sigma=0.1).phi
exc = Population(N=50000, tau=1e-2, phi=phi)

S, P = 1, 16
sequences = [GaussianSequence(P,exc.size,seed=i) for i in range(S)]
patterns = np.stack([s.inputs for s in sequences])

conn_EE = SparseConnectivity(source=exc, target=exc, p=0.005, seed=43, disable_pbar=True)
synapse = LinearSynapse(conn_EE.K, A=1)
conn_EE.store_sequences(patterns, synapse.h_EE)

net = RateNetwork(exc, c_EE=conn_EE, formulation=1)

r0 = exc.phi(patterns[0,0,:])
net.simulate(T, r0=r0)

M = np.mean(net.exc.state**2, axis=0)
overlaps = sequences[0].overlaps(net, exc)
correlations = sequences[0].overlaps(net, exc, correlation=True)
state = np.copy(net.exc.state)

[========================================================================] 100%
100%|██████████| 16/16 [00:06<00:00,  2.41it/s]


In [4]:
np.save("data/data_network.npy", [overlaps, M])